In [4]:
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 762.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 7.1 MB/s eta 0:00:0000:0100:01


In [1]:
import torchvision

In [5]:
data = torchvision.datasets.ImageNet("/Users/deepakdhull/data/imagenet", download=True)

RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in /Users/deepakdhull/data/imagenet.